In [54]:
import pandas as pd
import os

In [55]:
def get_trade_actions(prev_weights, new_weights, output_path="input/TradeDecisions.csv", tolerance=0.01):
    out = []

    for i in range(len(prev_weights)):
        ticker = prev_weights.loc[i, 'ticker']

        prev_weight = prev_weights.loc[i, 'weight_prev']
        new_weight = new_weights.loc[i, 'weight_new']

        if abs(new_weight - prev_weight) < tolerance:
            action = 'HOLD'
        else:
            if new_weight >= 0:
                if new_weight > prev_weight:
                    action = 'BUY' # increase long (and/or cover short if w_prev was negative)
                else:
                    action = 'SELL' # decrease long
            else:
                if prev_weight >= 0:
                    action = 'SHORT' # entering short
                elif new_weight < prev_weight:
                    action = 'SHORT' # increase short
                else:
                    action = 'COVER' # covering short

        out.append({
            'ticker': ticker,
            'prev_weight': prev_weight,
            'new_weight': new_weight,
            'action': action
        })

    df = pd.DataFrame(out)
    
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    
    return df

In [56]:
def prepare_execution_input(trade_decisions_path="input/TradeDecisions.csv",
                            market_data_path="data/market_data.csv",
                            output_path="input/input.csv",
                            portfolio_value=1_000_000):

    trades = pd.read_csv(trade_decisions_path)
    market = pd.read_csv(market_data_path) #not sure where we are pulling this from, might need update

    merged = trades.merge(market, on='ticker', how='left')

    missing = merged[merged[['exec_price','prev_price','volume']].isnull().any(axis=1)]
    if not missing.empty:
        print(missing)
        merged = merged.dropna(subset=['exec_price','prev_price','volume'])

    merged['shares'] = ((merged['new_weight'] - merged['prev_weight']) * portfolio_value / merged['exec_price']).abs().round()
    merged['shares'] = merged['shares'].fillna(0).astype(int)
                                                         
    exec_input = merged[['ticker', 'action', 'prev_price', 'exec_price', 'shares', 'volume']]

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    exec_input.to_csv(output_path, index=False)
    
    return exec_input

In [57]:
import pandas as pd

def execute_trades(input_path, output_path, slippage_rate=0.0005):
    df = pd.read_csv(input_path)
    
    results = []
    
    for _, row in df.iterrows():
        ticker = row['ticker']
        action = row['action']
        prev_price = row['prev_price']
        exec_price = row['exec_price']
        shares = row['shares']
        volume = row['volume']
        
        slippage = (shares / volume) * slippage_rate
        if slippage < 0.0001:  # Prevents slippage from being too small / defaulting to 0.0
            slippage = 0.0001  # Can adjust this threshold 
        
        cost = slippage * exec_price * shares
        
        if action == 'BUY':
            realized_PnL = (prev_price - exec_price) * shares - cost
        elif action == 'SELL':
            realized_PnL = (exec_price - prev_price) * shares - cost
        elif action == 'SHORT':
            realized_PnL = (prev_price - exec_price) * shares - cost
        elif action == 'COVER':
            realized_PnL = (exec_price - prev_price) * shares - cost
        else:
            realized_PnL = 0  # Can be edited for HOLD
        
        results.append({
            'ticker': ticker,
            'action': action,
            'exec_price': exec_price,
            'shares': shares,
            'slippage': round(slippage, 6),  
            'cost': round(cost, 2), 
            'realized_PnL': round(realized_PnL, 2)  
        })
    
    out_df = pd.DataFrame(results)
    
    out_df.to_csv(output_path, index=False)
    
    return out_df 


In [58]:
prev_weights = pd.DataFrame({
    'ticker': ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'NFLX', 'META', 'AMZN'],
    'weight_prev': [0.30, 0.40, 0.25, -0.10, -0.20, 0.0, -0.05]
})

new_weights = pd.DataFrame({
    'ticker': ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'NFLX', 'META', 'AMZN'],
    'weight_new': [0.42, 0.33, 0.25, -0.20, -0.10, 0.05, 0.05]
})

#get_trade_actions(prev_weights, new_weights)

market_data = pd.DataFrame({
    'ticker': ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'NFLX', 'META', 'AMZN', 'NVDA', 'BABA', 'AMD'],
    'prev_price': [185.3, 317.5, 145.2, 695.5, 505.4, 350.2, 3240.5, 754.3, 164.8, 111.3],
    'exec_price': [186.0, 316.0, 145.5, 700.0, 510.0, 355.0, 3250.0, 760.0, 167.0, 113.0],
    'volume': [1_200_000, 980_000, 800_000, 500_000, 650_000, 1_000_000, 1_200_000, 1_500_000, 850_000, 1_200_000]
})

os.makedirs("data", exist_ok=True)
market_data.to_csv("data/market_data.csv", index=False)

In [59]:
trades = get_trade_actions(prev_weights, new_weights)
exec_input = prepare_execution_input()

input_path = "input/input.csv"         
output_path = "output/outputs.csv"  

os.makedirs(os.path.dirname(output_path), exist_ok=True)
exec_output = execute_trades(input_path=input_path, output_path=output_path)

In [60]:
print("Market Data:")
print(market_data)
print("\nTrade Decisions:")
print(trades)
print("\nExecution Input:")
print(exec_input)
print("\nExecution Output:")
print(exec_output)

Market Data:
  ticker  prev_price  exec_price   volume
0   AAPL       185.3       186.0  1200000
1   MSFT       317.5       316.0   980000
2   GOOG       145.2       145.5   800000
3   TSLA       695.5       700.0   500000
4   NFLX       505.4       510.0   650000
5   META       350.2       355.0  1000000
6   AMZN      3240.5      3250.0  1200000
7   NVDA       754.3       760.0  1500000
8   BABA       164.8       167.0   850000
9    AMD       111.3       113.0  1200000

Trade Decisions:
  ticker  prev_weight  new_weight action
0   AAPL         0.30        0.42    BUY
1   MSFT         0.40        0.33   SELL
2   GOOG         0.25        0.25   HOLD
3   TSLA        -0.10       -0.20  SHORT
4   NFLX        -0.20       -0.10  COVER
5   META         0.00        0.05    BUY
6   AMZN        -0.05        0.05    BUY

Execution Input:
  ticker action  prev_price  exec_price  shares   volume
0   AAPL    BUY       185.3       186.0     645  1200000
1   MSFT   SELL       317.5       316.0     222